In [4]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize

In [2]:
interactions_df = pd.read_csv('lastfm_user_scrobbles.csv')
titles_df = pd.read_csv('lastfm_artist_list.csv')

In [3]:
# проверяем распределение
interactions_df.groupby('user_id').count().mean()

artist_id    49.044397
scrobbles    49.044397
dtype: float64

In [6]:
titles_df

,artist_id,artist_name
artist_id,,
1,1,__Max__
2,2,_Algol_
3,3,-123 Min.
4,4,-Oz-
5,5,-T De Sangre
...,...,...
17489,17489,鷺巣詩郎
17490,17490,黃立行
17491,17491,黄义达


In [17]:
# имена артистов
titles_df.index = titles_df['artist_id']
title_dict = titles_df['artist_name'].to_dict()

In [8]:
# Матрица UI 
# в первой переменной индексы юзеров, а во второй переменной их позиции
rows, r_pos = np.unique(interactions_df.values[:, 0], return_inverse = True)

In [11]:
# тоже самое, но теперь для исполнителей
cols, c_pos = np.unique(interactions_df.values[:, 1], return_inverse = True)

In [14]:
interactions_sparse = sparse.csr_matrix((interactions_df.values[:, 2], (r_pos, c_pos)))
interactions_sparse

<1892x17493 sparse matrix of type '<class 'numpy.int64'>'
	with 92723 stored elements in Compressed Sparse Row format>

In [15]:
# нормализуем и получаем матрицу схожести
Pui = normalize(interactions_sparse, norm = 'l2', axis = 1)
sim = Pui.T * Pui

In [16]:
# косинусная близость
sim.todense()

matrix([[0.00120405, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.00258841, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.00383689, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.0228355 , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.00827076,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.0010429 ]])

In [19]:
# получаем из матрицы нужную строчку (sim[]), конвертируем в array, 
# сортирумеи и берем последние N похожих
N = 10
[title_dict[i+1] for i in sim[10180].toarray().argsort()[0][-N:]]

['Metallica',
 'Satyricon',
 'Immortal',
 'Morbid Angel',
 'Tool',
 'Opeth',
 'Iron Maiden',
 'Him',
 'Moonspell',
 'Hirax']

In [22]:
# делаем предсказания для пользователей 
interactions_sparse_transposed = interactions_sparse.transpose(copy = True)
Piu = normalize(interactions_sparse_transposed, norm = 'l2', axis = 1)

fit = Pui * Piu * Pui
print(fit.shape)

(1892, 17493)


In [24]:
# группы, которые слушал пользователь 520
init_set = set([title_dict[i+1] for i in np.nonzero(interactions_sparse[520])[1].tolist()])

In [25]:
# что порекомендовали пользователю 520
pred_set = set([title_dict[i+1] for i in fit[520].toarray().argsort()[0][-20:].tolist()])

In [26]:
len(pred_set - init_set)

6